In [1]:
# Principais mudanças
    
# Baseline: 20200610-16h46--20200610-13h41
# Ver apresentação do dia 15/06/2020.

In [2]:
images_dimensions = 128
attribute = 'initial-set'
totalInputImages = 15000
BATCH_SIZE = 50
latent_dim = 250

In [3]:
EPOCHS = 200

In [4]:
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot

# Daniel
import os
import sys
import numpy as np
from PIL import Image
from tqdm import tqdm
import cv2
import time
from keras.initializers import RandomNormal
from numpy.random import random
from numpy.random import choice
import shutil

Using TensorFlow backend.


In [5]:
import warnings
warnings.filterwarnings( 'ignore' )

In [6]:
import tensorflow as tf
print( tf.__version__ )
print( 'Num GPUs Available: ', len( tf.config.list_physical_devices( 'GPU' ) ) )

2.1.0
Num GPUs Available:  1


In [7]:
lastTimeNotebook = time.time()

In [8]:
IMAGES_FOLDER = './images/'
folderDimensionsSufix = str( images_dimensions ) + 'x' + str( images_dimensions )

padronizedFolder = f'{IMAGES_FOLDER}padronized--{attribute}--{folderDimensionsSufix}--{totalInputImages}--bs-{BATCH_SIZE}--ls-{latent_dim}/'

In [9]:
trainedModelFolder = f'./trained-model--{attribute}--{folderDimensionsSufix}--{totalInputImages}--bs-{BATCH_SIZE}--ls-{latent_dim}/'
generatorFolder = trainedModelFolder + 'generator/'
generatedFolder = trainedModelFolder + 'generated/'

In [10]:
try:
    shutil.rmtree(trainedModelFolder)
    os.mkdir(trainedModelFolder)
except OSError as e:
    os.mkdir(trainedModelFolder)

In [11]:
try:
    shutil.rmtree(generatorFolder)
    os.mkdir(generatorFolder)
except OSError as e:
    os.mkdir(generatorFolder)

In [12]:
try:
    shutil.rmtree(generatedFolder)
    os.mkdir(generatedFolder)
except OSError as e:
    os.mkdir(generatedFolder)

In [13]:
def define_discriminator(in_shape=(128,128,1)):
    model = Sequential()
    # normal
    model.add(Conv2D(128, (6,6), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))

    # downsample to 64x64
    model.add(Conv2D(128, (6,6), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    
    # downsample to 32x32
    model.add(Conv2D(128, (6,6), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    
    # downsample to 16x16
    model.add(Conv2D(128, (6,6), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    
    # downsample to 8x8
    model.add(Conv2D(128, (6,6), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    
    # classifier
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    print()
    print('=================================================================')
    print('========================= Discriminator =========================')
    print('=================================================================')
    print()
    print( model.summary() )

    return model

In [14]:
# define the standalone generator model
def define_generator(latent_dim):
    model = Sequential()
    
    # foundation for 8x8 feature maps
    n_nodes = 128 * 8 * 8
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Reshape((8, 8, 128)))
    
    # upsample to 16x16
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    
    # upsample to 32x32
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    
    # upsample to 64x64
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    
    # upsample to 128x128
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    
    # output layer 128x128x3
    model.add(Conv2D(1, (5,5), activation='tanh', padding='same'))

    print()
    print('=================================================================')
    print('=========================== Generator ===========================')
    print('=================================================================')
    print()
    print( model.summary() )
    
    return model

In [15]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(g_model)
    # add the discriminator
    model.add(d_model)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [16]:
# load and prepare training images
def load_real_samples():
    
    _, _, files = next( os.walk( padronizedFolder ) )

    images = []
    for file in tqdm(files):

        image = cv2.imread( padronizedFolder + file, cv2.IMREAD_GRAYSCALE )
        image = image[..., np.newaxis]
        image = (image - 127.5) / 127.5
        images.append( image )

    images = np.asarray(images)
    return images

In [17]:
# Daniel # 5.5.3 Use Label Smoothing

# example of smoothing class=1 to [0.7, 1.2]
def smooth_positive_labels(y):
    return y - 0.3 + (random(y.shape) * 0.5)

# example of smoothing class=0 to [0.0, 0.3]
def smooth_negative_labels(y):
    return y + random(y.shape) * 0.3

In [18]:
# Daniel # 5.5.4 Use Noisy Labels

# randomly flip some labels
def noisy_labels(y, p_flip):
    
    # determine the number of labels to flip
    n_select = int(p_flip * y.shape[0])
    
    # choose labels to flip
    flip_ix = choice([i for i in range(y.shape[0])], size=n_select)
    
    # invert the labels in place
    y[flip_ix] = 1 - y[flip_ix]
    
    return y

In [19]:
# select real samples
def generate_real_samples(dataset, n_samples):
    
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    
    # retrieve selected images
    X = dataset[ix]
    
    # generate 'real' class labels (1)
    y = ones((n_samples, 1))

    # Daniel # 5.5.3 Use Label Smoothing
    # smooth labels
    y = smooth_positive_labels(y)

    # Daniel # 5.5.4 Use Noisy Labels
    # flip labels with 5% probability
    y = noisy_labels(y, 0.05)

    return X, y

In [20]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

In [21]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
    
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    
    # predict outputs
    X = g_model.predict(x_input)
    
    # create 'fake' class labels (0)
    y = zeros((n_samples, 1))

    # Daniel # 5.5.3 Use Label Smoothing
    # smooth labels
    y = smooth_negative_labels(y)

    # Daniel # 5.5.4 Use Noisy Labels
    # flip labels with 5% probability
    y = noisy_labels(y, 0.05)

    return X, y

In [22]:
# create and save a plot of generated images
def save_plot(examples, epoch, n=10):
    
    # Daniel
    examples = ( examples * 127.5 ) + 127.5
    epochFolder = generatedFolder + f'epoch-{epoch + 1}/'
    
    # Daniel
    for i in range(0, 30) :
        
        cv2.imwrite( 
            epochFolder + f'epoch-{epoch + 1}--{i + 1}.png', 
            examples[i] )

In [23]:
# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
    
    # prepare real samples
    X_real, y_real = generate_real_samples(dataset, n_samples)
    
    # evaluate discriminator on real examples
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    
    # evaluate discriminator on fake examples
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    
    # save plot
    save_plot(x_fake, epoch)
    
    # save the generator model tile file
    filename = generatorFolder + 'generator_model_%03d.h5' % (epoch + 1)
    g_model.save(filename)

In [24]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=128):
    
    lastTime = time.time()
    
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    
    # manually enumerate epochs
    for i in range(n_epochs):
        
        print()
        
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            
            # Time
            now = time.time()
            interval = now - lastTime
            lastTime = now
            
            # get randomly selected 'real' samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            
            # update discriminator model weights
            d_loss1, _ = d_model.train_on_batch(X_real, y_real)
            
            # generate 'fake' examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            
            # update discriminator model weights
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
            
            # prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            
            # create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
            
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            
            # summarize loss on this batch
            print('> epoch=%d, batch=%d/%d, d1=%.3f, d2=%.3f, g=%.3f, time=%.2f sec' %
                (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss, (interval) ))
            
        # evaluate the model performance, sometimes
        if (i + 1) % 1 == 0:

            # Daniel
            os.mkdir( generatedFolder + f'epoch-{i + 1}/' )
            
            summarize_performance(i, g_model, d_model, dataset, latent_dim)

In [ ]:
print( 'Processing...' )
print()

# create the discriminator
d_model = define_discriminator()

# create the generator
g_model = define_generator(latent_dim)

# create the gan
gan_model = define_gan(g_model, d_model)

# load image data
dataset = load_real_samples()

# train model
train(
    g_model, 
    d_model, 
    gan_model, 
    dataset, 
    latent_dim, 
    n_epochs=EPOCHS, 
    n_batch=BATCH_SIZE)

print( '\n----------------------------------------------------' )
print( 'Finalized' )

# Time
now = time.time()
interval = now - lastTimeNotebook
print('Notebook Time: %.2f min' % (interval/60))

Processing...


========================= Discriminator =========================

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 128)     4736      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128, 128, 128)     0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 128)       589952    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 128)       589952    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 32, 32, 128)       0         
_____________________________________

100%|██████████| 15000/15000 [05:12<00:00, 47.92it/s]



> epoch=1, batch=1/300, d1=0.692, d2=0.695, g=0.693, time=0.00 sec
> epoch=1, batch=2/300, d1=0.612, d2=0.697, g=0.691, time=7.90 sec
> epoch=1, batch=3/300, d1=0.379, d2=0.720, g=0.669, time=0.44 sec
> epoch=1, batch=4/300, d1=0.227, d2=0.726, g=0.688, time=0.44 sec
> epoch=1, batch=5/300, d1=0.289, d2=0.732, g=0.681, time=0.44 sec
> epoch=1, batch=6/300, d1=0.276, d2=0.733, g=0.732, time=0.44 sec
> epoch=1, batch=7/300, d1=0.502, d2=0.688, g=0.708, time=0.44 sec
> epoch=1, batch=8/300, d1=0.449, d2=0.688, g=0.716, time=0.44 sec
> epoch=1, batch=9/300, d1=0.221, d2=0.680, g=0.747, time=0.44 sec
> epoch=1, batch=10/300, d1=0.413, d2=0.674, g=0.754, time=0.44 sec
> epoch=1, batch=11/300, d1=0.513, d2=0.671, g=0.737, time=0.44 sec
> epoch=1, batch=12/300, d1=0.500, d2=0.661, g=0.792, time=0.44 sec
> epoch=1, batch=13/300, d1=0.360, d2=0.617, g=1.050, time=0.44 sec
> epoch=1, batch=14/300, d1=0.301, d2=0.507, g=1.954, time=0.44 sec
> epoch=1, batch=15/300, d1=0.269, d2=0.560, g=1.336, ti